In [26]:
import re
from functools import lru_cache
from typing import List, Optional

from rapidfuzz import process, fuzz
import pandas as pd

# -------------------------
# Кардинальные
# -------------------------
UNITS = {
    "ноль": 0,
    "один": 1, "одна": 1, "одно": 1,
    "два": 2, "две": 2,
    "три": 3, "четыре": 4, "пять": 5, "шесть": 6, "семь": 7, "восемь": 8, "девять": 9,
    "двух": 2, "трех": 3, "четырех": 4, "пяти": 5, "шести": 6, "семи": 7, "восьми": 8, "девяти": 9,
    "одного": 1, "одному": 1, "одним": 1, "одни": 1,
    "двум": 2, "двух": 2,
    "трем": 3, "трех": 3,
    "четырем": 4, "четырех": 4,
    "пятью": 5, "шестью": 6, "семью": 7, "восьмью": 8, "девятью": 9,
    "двое": 2, "трое": 3, "четверо": 4, "пятеро": 5
}

TEENS = {
    "десять": 10, "одиннадцать": 11, "двенадцать": 12, "тринадцать": 13, "четырнадцать": 14,
    "пятнадцать": 15, "шестнадцать": 16, "семнадцать": 17, "восемнадцать": 18, "девятнадцать": 19,
    "десяти": 10, "одиннадцати": 11, "двенадцати": 12, "тринадцати": 13, "четырнадцати": 14,
    "пятнадцати": 15, "шестнадцати": 16, "семнадцати": 17, "восемнадцати": 18, "девятнадцати": 19,

}

TENS = {
    "двадцать": 20, "тридцать": 30, "сорок": 40, "пятьдесят": 50, "шестьдесят": 60,
    "семьдесят": 70, "восемьдесят": 80, "девяносто": 90,
    "двадцати": 20, "тридцати": 30, "сорока": 40, "пятидесяти": 50, "шестидесяти": 60,
    "семидесяти": 70, "восьмидесяти": 80, "девяноста": 90, "двадцати": 20,
    "тридцати": 30,

}

HUNDREDS = {
    "сто": 100, "двести": 200, "триста": 300, "четыреста": 400, "пятьсот": 500, "шестьсот": 600,
    "семьсот": 700, "восемьсот": 800, "девятьсот": 900,
    "ста": 100, "двухсот": 200, "трехсот": 300, "четырехсот": 400, "пятисот": 500,
    "шестисот": 600, "семисот": 700, "восьмисот": 800, "девятисот": 900,

}

# -------------------------
# Масштабы
# -------------------------
SCALES = {
    "тысяча": 1_000, "тысячи": 1_000, "тысяч": 1_000, "тыс": 1_000,
    "миллион": 1_000_000, "миллиона": 1_000_000, "миллионов": 1_000_000, "млн": 1_000_000,
    "миллиард": 1_000_000_000, "миллиарда": 1_000_000_000, "миллиардов": 1_000_000_000, "млрд": 1_000_000_000,
    "тыщ": 1_000, "тыщи": 1_000, "тысячу": 1000,
    "тыща": 1000,
    "тыщи": 1000, 
}

# -------------------------
# Порядковые (частые формы + падежи)
# -------------------------
ORD_UNITS = {
    "первый": 1, "первая": 1, "первого": 1, "первой": 1, "первому": 1,
    "второй": 2, "вторая": 2, "второго": 2, "второму": 2, "второе": 2,
    "третий": 3, "третья": 3, "третьего": 3, "третьему": 3,
    "четвертый": 4, "четвертого": 4,
    "пятый": 5, "пятого": 5,
    "шестой": 6, "шестого": 6,
    "седьмой": 7, "седьмого": 7,
    "восьмой": 8, "восьмого": 8,
    "девятый": 9, "девятого": 9,
    "десятый": 10, "десятого": 10,
    "одиннадцатый": 11, "одиннадцатого": 11,
    "двенадцатый": 12, "двенадцатого": 12,
    "тринадцатый": 13, "тринадцатого": 13,
    "четырнадцатый": 14, "четырнадцатого": 14,
    "пятнадцатый": 15, "пятнадцатого": 15,
    "шестнадцатый": 16, "шестнадцатого": 16,
    "семнадцатый": 17, "семнадцатого": 17,
    "восемнадцатый": 18, "восемнадцатого": 18, "восемнадцатое": 18,
    "девятнадцатый": 19, "девятнадцатого": 19,
    "двадцатый": 20, "двадцатого": 20,
    "тридцатый": 30, "тридцатого": 30,
    "первую": 1,
    "вторую": 2,
    "третью": 3,
    "четвертую": 4,    
    "тринадцатое": 13,
    "двухсотую": 200, 
    "двухстую": 200,  

}

# Для составных порядковых: "двадцать третьего" => 20 + 3
ORD_TENS = {"двадцать": 20, "тридцать": 30}

FORBIDDEN_TOKENS = {
    "срок", "срока", "сроки", "срочно",
    "стоимость", "стоимости",
}


# Коннекторы, которые могут появляться внутри числового выражения
CONNECTORS = {"и", "да"}

ASR_FIX = {
    "двеси": "двести",
    "двес": "двести",
    "дваста": "двести",
    "двиста": "двести",
    "двисти": "двести",
    "двесте": "двести",
}


# Единый словарь канонических слов
CANON = {}
CANON.update(UNITS)
CANON.update(TEENS)
CANON.update(TENS)
CANON.update(HUNDREDS)
CANON.update(SCALES)
CANON.update(ORD_UNITS)
CANON.update(ORD_TENS)

CANON_KEYS = list(CANON.keys())


In [27]:
_re_nonletters = re.compile(r"[^а-яa-z0-9]+", re.IGNORECASE)

def _norm_token(token: str) -> str:
    """
    Делает токен более устойчивым к шуму ASR:
    - lowercase, ё->е
    - удаление мусорных символов
    - сжатие слишком длинных повторов букв
    """
    t = token.lower().replace("ё", "е")
    t = _re_nonletters.sub("", t)
    t = re.sub(r"(.)\1{2,}", r"\1\1", t)  # 'пяяять' -> 'пяять'
    return t



CANON_KEYS_LONG = [k for k in CANON_KEYS if len(k) >= 4]

@lru_cache(maxsize=200_000)
def _fuzzy_to_canon(token_norm: str, min_score: int) -> Optional[str]:
    if not token_norm:
        return None
    if token_norm in CANON:
        return token_norm
    if token_norm in ASR_FIX:
        return ASR_FIX[token_norm]

    # склейки
    if token_norm.startswith("тыс"):
        return "тыс"
    if token_norm.startswith("млн"):
        return "млн"
    if token_norm.startswith("млрд"):
        return "млрд"

    candidates = CANON_KEYS_LONG if len(token_norm) >= 5 else CANON_KEYS
    match = process.extractOne(token_norm, candidates, scorer=fuzz.WRatio)
    if not match:
        return None

    key, score = match[0], match[1]
    if score < min_score:
        return None

    # фильтры против ложных матчей
    if abs(len(token_norm) - len(key)) > 3:
        return None
    if token_norm[0] != key[0]:
        return None

    return key


def _is_number_word(token: str, min_score: int) -> bool:
    tn = _norm_token(token)
    if not tn or tn.isdigit():
        return False

    if tn in CONNECTORS:
        return False  # коннекторы учитываем только внутри спана

    if tn in CANON:
        return True

    if len(tn) < 4:
        return False

    c = _fuzzy_to_canon(tn, min_score)
    return c is not None and c not in CONNECTORS




In [28]:
def parse_cardinal(canon_tokens: List[str]) -> Optional[int]:
    """
    Кардинальные числа:
    - current копит значение до масштаба
    - при масштабе (тыс/млн/...) умножаем и переносим в total
    """
    total = 0
    current = 0

    for t in canon_tokens:
        if t in CONNECTORS:
            continue
        if t in UNITS:
            current += UNITS[t]
        elif t in TEENS:
            current += TEENS[t]
        elif t in TENS:
            current += TENS[t]
        elif t in HUNDREDS:
            current += HUNDREDS[t]
        elif t in SCALES:
            scale = SCALES[t]
            if current == 0:
                current = 1  # "тысяча" == 1000
            total += current * scale
            current = 0
        else:
            return None

    return total + current


In [29]:
def parse_ordinal(canon_tokens: List[str]) -> Optional[int]:
    """
    Порядковые:
    - 1 токен из ORD_UNITS -> число
    - 2 токена: десятки (ORD_TENS) + единицы (ORD_UNITS) -> сумма
      пример: "двадцать третьего" -> 23
    """
    tokens = [t for t in canon_tokens if t not in CONNECTORS]
    if not tokens:
        return None

    if len(tokens) == 1 and tokens[0] in ORD_UNITS:
        return ORD_UNITS[tokens[0]]

    if len(tokens) == 2 and tokens[0] in ORD_TENS and tokens[1] in ORD_UNITS:
        base = ORD_TENS[tokens[0]]
        tail = ORD_UNITS[tokens[1]]
        if 1 <= tail <= 9:
            return base + tail

    return None


In [30]:
def split_or_sum(canon_tokens: List[str]) -> Optional[List[int]]:
    """
    Возвращает список чисел для вставки в текст.
    Сначала обрабатываем 'диктовку' (перечисления), потом порядковые, потом обычный парсинг кардинальных.
    """
    has_scale = any(t in SCALES for t in canon_tokens)
    sig = [t for t in canon_tokens if t not in CONNECTORS]

    # ---------- 1) ДИКТОВКА / ПЕРЕЧИСЛЕНИЯ (важно делать ДО parse_cardinal) ----------

    # (1a) "двести триста" -> "200 300"
    if not has_scale and len(sig) >= 2 and all(t in HUNDREDS for t in sig):
        return [HUNDREDS[t] for t in sig]

    # (1b) "один два три" / "семь и восемь" -> "1 2 3" / "7 8"
    if not has_scale and len(sig) >= 2 and all(t in UNITS for t in sig):
        return [UNITS[t] for t in sig]

    # (1c) "сорок восемьдесят" -> "40 80"
    if not has_scale and len(sig) >= 2 and all(t in TENS for t in sig):
        return [TENS[t] for t in sig]

    # (1d) "четыре семьсот" -> "4 700" ; "три пятьсот" -> "3 500"
    if not has_scale and len(sig) == 2 and (sig[0] in UNITS) and (sig[1] in HUNDREDS):
        return [UNITS[sig[0]], HUNDREDS[sig[1]]]

    # ---------- 2) ПОРЯДКОВЫЕ ----------
    sig = [t for t in canon_tokens if t not in CONNECTORS]

# кардинал + порядковая единица -> одно число: "двести восемьдесят пятого" -> 285
    if len(sig) >= 2 and sig[-1] in ORD_UNITS and 1 <= ORD_UNITS[sig[-1]] <= 9:
        base = parse_cardinal(sig[:-1])
        if base is not None:
            return [base + ORD_UNITS[sig[-1]]]

    ord_val = parse_ordinal(canon_tokens)
    if ord_val is not None:
        return [ord_val]

    # ---------- 3) ОБЫЧНОЕ КАРДИНАЛЬНОЕ ЧИСЛО ----------
    card_val = parse_cardinal(canon_tokens)
    if card_val is None:
        return None
    return [card_val]


In [31]:
def normalize_numbers(text: str, *, min_score: int = 94) -> str:
    tokens = text.split()
    out: List[str] = []
    i = 0
    max_span_len = 12

    while i < len(tokens):
        DISALLOW_SCALE_START = {"тыщ", "тыс", "млн", "млрд"}
        ALLOW_SCALE_START = {"тысяча", "тысячу", "тысячи", "тысяч", "тыща", "тыщи"}
        tn0 = _norm_token(tokens[i])
        c0 = _fuzzy_to_canon(tn0, min_score)
        
        if c0 in DISALLOW_SCALE_START:
            out.append(tokens[i])
            i += 1
            continue

        if not _is_number_word(tokens[i], min_score):
            out.append(tokens[i])
            i += 1
            continue

        j = i
        canon: List[str] = []
        last_was_number = False

        while j < len(tokens) and (j - i) < max_span_len:
            tn = _norm_token(tokens[j])

            # коннектор только между числами
            if tn in CONNECTORS:
                if last_was_number and (j + 1 < len(tokens)) and _is_number_word(tokens[j + 1], min_score):
                    canon.append(tn)
                    j += 1
                    last_was_number = False
                    continue
                break

            if not _is_number_word(tokens[j], min_score):
                break

            c = _fuzzy_to_canon(tn, min_score)
            if c is None or c in CONNECTORS:
                break

            canon.append(c)
            last_was_number = True
            j += 1

        nums = split_or_sum(canon)
        if not nums:
            out.append(tokens[i])
            i += 1
            continue

        # сохраняем "и" между двумя числами (как было у тебя на 0.916)
        if "и" in canon and len(nums) == 2:
            out.extend([str(nums[0]), "и", str(nums[1])])
        else:
            out.extend(str(n) for n in nums)

        i = j

    return " ".join(out)


In [32]:
# calibration
import pyarrow.feather as feather

cal = feather.read_table("calibration.f").to_pandas()
cal["pred"] = cal["task_text"].apply(lambda s: normalize_numbers(s, min_score=94))
acc = (cal["pred"] == cal["ground_truth"]).mean()
print("Calibration accuracy:", acc)

# посмотреть ошибки
errors = cal.loc[cal["pred"] != cal["ground_truth"], ["task_text", "ground_truth", "pred"]]
print("Errors:", len(errors))
print(errors.head(20).to_string(index=False))

# test -> answer.csv
test = feather.read_table("test.f").to_pandas()
test["answer"] = test["task_text"].apply(lambda s: normalize_numbers(s, min_score=94))
test.to_csv("answer.csv", index=False)
print("Saved answer.csv")


Calibration accuracy: 0.96
Errors: 20
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  